In [66]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import pandas as pd
import math
import fetcher
import datetime
from datetime import timedelta
import warnings; warnings.simplefilter('ignore')

# Sample data for multiple stocks
startDate = datetime.date(2020, 1, 1)
#endDate = datetime.date(2024, 8, 20)
endDate = datetime.date.today()
stocks = ['CDNA', 'VCYT', 'PRCT', 'NTRA', 'GMED', 'TGTX', 'MIRM', 'ALNY', 'LLY', 'UTHR', 'RKLB', 'AXON', 'CR', 'NVDA', 'MPWR', 'AVGO', 'FN', 'TSM', 'COHR', 'KLAC', 'NVMI', 'ITRI', 'PSTG', 'AAPL', 'ASPN', 'PPC', 'CDE', 'IAG', 'KGC', 'AGI', 'EGO', 'CRS', 'KNF', 'LPX', 'HA', 'UBER', 'KEX', 'GGAL', 'SSB', 'ALLY', 'JXN', 'TPG', 'URI', 'PIPR', 'MC', 'PJT', 'EVR', 'GS', 'ZION', 'CADE', 'FULT', 'BAC', 'PLTR', 'MNDY', 'VERX', 'PANW', 'FICO', 'NOW', 'WIX', 'SPOT', 'META', 'NFLX', 'GOOGL', 'GOOG', 'FIX', 'MTZ', 'PRIM', 'DY', 'MHO', 'CVCO', 'SN', 'FYBR', 'ATGE', 'LRN', 'HRB', 'BKNG', 'BCO', 'SFM', 'MELI', 'AMZN', 'BABA', 'HD']


print(len(stocks))
# Calculate the number of rows needed (3 plots per row)
num_stocks = len(stocks)
num_cols = 3
num_rows = math.ceil(num_stocks / num_cols)

# Create subplots with dynamic rows and columns
def subPlot_titles(): #### THis looks complex coz I was trying to name the charts with the stock names as well as volumes to the corresponding chart below them
    titles = []
    for i in range(len(stocks)*2):
        if len(stocks)%num_cols == 0:

            if math.floor(i/num_cols)%2 ==0:
                titles.append(stocks[i-int(num_cols*math.floor(i/num_cols)/2)])
            else:
                df = fetcher.__download_data( [titles[i - num_cols]], startDate, endDate, "US")
                Vol = df["Volume"].values[-1]
                AVol = df['Volume'].rolling(window=30).mean().values[-1]
                RVol = Vol/AVol
                titles.append("Vol:{0}, AVol:{1:.2f}, RVol:{2:.2f}".format(Vol, AVol, RVol))
        else:
            if i < len(stocks)*2-1:
                if math.floor(i/num_cols)%2 ==0 :
                    titles.append(stocks[i-int(num_cols*math.floor(i/num_cols)/2)])
                else:
                    df = fetcher.__download_data( [titles[i - num_cols]], startDate, endDate, "US")
                    Vol = df["Volume"].values[-1]
                    AVol = df['Volume'].rolling(window=30).mean().values[-1]
                    RVol = Vol/AVol
                    titles.append("Vol:{0}, AVol:{1:.2f}, RVol:{2:.2f}".format(Vol, AVol, RVol))
            else: 
                titles = titles + ["NULL"]*(num_cols-len(stocks)%num_cols)
                i += (num_cols-len(stocks)%num_cols)
                df = fetcher.__download_data( [titles[i - num_cols]], startDate, endDate, "US")
                Vol = df["Volume"].values[-1]
                AVol = df['Volume'].rolling(window=30).mean().values[-1]
                RVol = Vol/AVol
                titles.append("Vol:{0}, AVol:{1:.2f}, RVol:{2:.2f}".format(Vol, AVol, RVol))

    return  titles


fig = make_subplots(
    rows=num_rows*2, 
    cols=num_cols, 
    shared_yaxes=False, 
    subplot_titles=subPlot_titles(),
    horizontal_spacing = 0.02,
    vertical_spacing=0.003
)


# Add candlestick traces and moving averages using a loop
for i, ticker in enumerate(stocks):
    # Calculate the moving averages
    df = fetcher.__download_data( [ticker], startDate, endDate, "US")
    df['Date'] = df.index
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    df['SMA_200'] = df['Close'].rolling(window=200).mean()
    df['Vol_SMA_30'] = df['Volume'].rolling(window=30).mean()
    df = df[-63:]

    row = math.floor(i / num_cols)*2 + 1
    col = (i % num_cols) + 1
    
    # Add candlestick trace
    fig.add_trace(go.Candlestick(
        x=df['Date'],
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        showlegend=False
        #name=df['Name'][0]
    ), row=row, col=col)
    
    # Add 10-day EMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['EMA_10'], 
        mode='lines',
        line=dict(color='blue', width=1),
        showlegend=False
        #name='10-day EMA'
    ), row=row, col=col)
    
    # Add 21-day EMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['EMA_21'], 
        mode='lines',
        line=dict(color='orange', width=1),
        showlegend=False
        #name='21-day EMA'
    ), row=row, col=col)
    
    # Add 50-day SMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['SMA_50'], 
        mode='lines',
        line=dict(color='green', width=1),
        showlegend=False
        #name='50-day SMA'
    ), row=row, col=col)

    # Add 200-day SMA trace
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['SMA_200'], 
        mode='lines',
        line=dict(color='red', width=1),
        showlegend=False
        #name='50-day SMA'
    ), row=row, col=col)

    # Add volume bar trace in the row below the candlesticks
    fig.add_trace(go.Bar(
        x=df['Date'], 
        y=df['Volume'], 
        marker_color='gray',
        showlegend=False
    ), row=row+1, col=col)

    # Add volume SMA trace in the row below the candlesticks
    fig.add_trace(go.Scatter(
        x=df['Date'], 
        y=df['Vol_SMA_30'], 
        marker_color='blue',
        showlegend=False
    ), row=row+1, col=col)

# Update layout for independent zooming and panning and to remove rangesliders
for i in range(1, num_rows * num_cols * 2 + 1):
    fig.update_layout({
        f'xaxis{i}_rangeslider_visible': False
    })
alldays =set(df.Date[0]+timedelta(x) for x in range((df.Date[len(df.Date)-1]- df.Date[0]).days))
missing=sorted(set(alldays)-set(df.Date))
fig.update_xaxes(rangebreaks=[dict(values=missing)])
fig.update_layout(
    title="Stock Candlestick Charts with Moving Averages",
    yaxis_title="Price",
    hovermode='closest',  # Ensure hover only shows info for the nearest point
    height=500*num_rows,  # Adjust height dynamically based on the number of rows
    width=1700,
    autosize=True,  # Enable autosizing
    dragmode='pan',  # Enables panning with left-click drag
)

# Enable scroll zoom and remove the box zoom option from the mode bar
config = dict(
    scrollZoom=True,  # Enable zooming with mouse scroll
    displayModeBar=True,  # Show the mode bar for interactivity options
    modeBarButtonsToRemove=['zoom2d'],  # Remove the zoom box tool
)

# Display the plot with the specified configuration
fig.show(config=config)


82


In [27]:
["dumb"]*(3-len(stocks)%3)

['dumb', 'dumb']

In [24]:
["a", "b"] + ["Null"]*(3-len(stocks)%3)

['a', 'b', 'Null', 'Null']